In [1]:
%matplotlib qt

In [2]:
import os, pyxdf, json, yaml
import mne
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold
from sklearn.pipeline import make_pipeline

# pyriemann import
from pyriemann.classification import MDM, TSclassifier
from pyriemann.estimation import Covariances
from pyriemann.utils.distance import distance

# sklearn imports
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import Pipeline
from mne.decoding import CSP
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC

import seaborn as sns

### Settings

In [3]:
config_file = 'config_MI-hands.yaml'

with open(config_file) as f:
    config = yaml.load(f.read(), Loader=yaml.Loader)
    print(config)
    locals().update(config)

lslDir = os.path.join(os.path.expanduser('~'), 'Documents\CurrentStudy')

{'subject': 'P00J', 'session': 2, 'task': 'MI-push', 'eeg_stream_type': 'EXG', 'markers_stream_type': 'Marker', 'default_ch_names': ['Fz', 'FC1', 'FC2', 'C3', 'Cz', 'C4', 'CP1', 'CP2', 'P3', 'Pz', 'P4', 'PO3', 'PO4', 'O1', 'Oz', 'O2'], 'montage_file': 'openbci_montage.elc', 'event_dict': {'rest': 0, 'MI/push': 1}, 'rest_duration': 8, 'task_duration': 8, 'tmin': 0.0, 'tmax': 8.0, 'window_size': 8.0, 'window_overlap': 0.0, 'plotGraphs': False, 'scalings': {'eeg': 0.0002}, 'plot_duration': 10, 'bp_l_freq': 1.0, 'bp_h_freq': 40.0, 'performICA': False, 'features': 'psd', 'tfr_type': 'morlet', 'baseline': (0.0, 0.1), 'vmin': -1.0, 'vmax': 1.0}


### Find LSL Files

In [4]:
# Find files
xdf_files = []
hasSubject = subject!=''
hasSession = session!=''
hasTask = task!=''
for root, dir, files in os.walk(lslDir):
    for file in files:
        validFile = True
        if hasSubject:
            validFile = validFile and (('sub-'+subject) in file)
        if hasSession:
            validFile = validFile and (('ses-S' + str(session).zfill(3)) in file)
        if hasTask:
            validFile = validFile and (('task-' + task) in file)
        validFile = validFile and file.endswith('.xdf')
        if validFile:
            print(file)
            matchingFile = os.path.join(root, file)
            xdf_files.append(matchingFile)

if len(xdf_files) == 0:
    print('No files found')

sub-P00J_ses-S002_task-MI-push_run-001_eeg.xdf
sub-P00J_ses-S002_task-MI-push_run-002_eeg.xdf
sub-P00J_ses-S002_task-MI-push_run-005_eeg.xdf
sub-P00J_ses-S002_task-MI-push_run-006_eeg.xdf
sub-P00J_ses-S002_task-MI-push_run-007_eeg.xdf


In [5]:
# Parse streams
eeg_stream, marker_stream = [], []

print('Parsing streams')
for xdf_file in xdf_files:
    streams, header = pyxdf.load_xdf(xdf_file)
    for i in range(len(streams)):
        if streams[i]['info']['type'][0] == eeg_stream_type:
            print("Found %s stream in %s" % (eeg_stream_type, os.path.basename(xdf_file)))
            eeg_stream.append(streams[i])
        elif streams[i]['info']['type'][0] == markers_stream_type:
            print("Found %s stream in %s" % (markers_stream_type, os.path.basename(xdf_file)))
            marker_stream.append(streams[i])
del streams, header

Parsing streams
Found EXG stream in sub-P00J_ses-S002_task-MI-push_run-001_eeg.xdf
Found Marker stream in sub-P00J_ses-S002_task-MI-push_run-001_eeg.xdf
Found Marker stream in sub-P00J_ses-S002_task-MI-push_run-002_eeg.xdf
Found EXG stream in sub-P00J_ses-S002_task-MI-push_run-002_eeg.xdf
Found Marker stream in sub-P00J_ses-S002_task-MI-push_run-005_eeg.xdf
Found EXG stream in sub-P00J_ses-S002_task-MI-push_run-005_eeg.xdf
Found Marker stream in sub-P00J_ses-S002_task-MI-push_run-006_eeg.xdf
Found EXG stream in sub-P00J_ses-S002_task-MI-push_run-006_eeg.xdf
Found EXG stream in sub-P00J_ses-S002_task-MI-push_run-007_eeg.xdf
Found Marker stream in sub-P00J_ses-S002_task-MI-push_run-007_eeg.xdf


### Extract EEG and Marker data

In [6]:
# Extract EEG Info
print("Extracting EEG info...")

ch_names = []
if eeg_stream[0]['info']['desc'][0]:
    print("EEG channel names found")
    for i in range(len(eeg_stream[0]['info']['desc'][0]['channels'][0]['channel'])):
        ch_names.append(eeg_stream[0]['info']['desc'][0]['channels'][0]['channel'][i]['label'][0])
else:
    ch_names = default_ch_names
print('Channels: ', ch_names)

sfreq = float(eeg_stream[0]['info']['nominal_srate'][0])
print('Sampling frequency: ', sfreq)

# Create MNE info object
eeg_info = mne.create_info(ch_names, sfreq, ch_types='eeg')

Extracting EEG info...
Channels:  ['Fz', 'FC1', 'FC2', 'C3', 'Cz', 'C4', 'CP1', 'CP2', 'P3', 'Pz', 'P4', 'PO3', 'PO4', 'O1', 'Oz', 'O2']
Sampling frequency:  125.0


In [7]:
# Setup Montage
montage = mne.channels.read_custom_montage(montage_file)
# montage.plot()

In [8]:
# Get all EEG data
eeg_raw_list = []

for n in range(len(eeg_stream)):
    # Create MNE Raw object
    eeg_data = np.transpose(eeg_stream[n]['time_series'])
    eeg_data = eeg_data / 1e6
    print(eeg_data.shape)
    eeg_raw = mne.io.RawArray(eeg_data, eeg_info)
    
    # Set montage
    eeg_raw = eeg_raw.set_montage(montage)

    # Add annotations
    onset, duration, description = [], [], []
    current_target = -1
    current_flash = -1
    for i in range(len(marker_stream[n]['time_series'])):
        if 'MI' in task:
            if ('rest' in marker_stream[n]['time_series'][i][0]) and ('cue' not in marker_stream[n]['time_series'][i][0]):
                window_onset = tmin
                window_start = marker_stream[n]['time_stamps'][i] - eeg_stream[n]['time_stamps'][0]
                while (window_onset + window_size <= tmax):
                    onset.append(window_start + window_onset)
                    duration.append(window_size)
                    description.append(marker_stream[n]['time_series'][i][0])
                    window_onset = window_onset + window_size - window_overlap
            elif ('task' in marker_stream[n]['time_series'][i][0]) and ('cue' not in marker_stream[n]['time_series'][i][0]):
                window_onset = tmin
                window_start = marker_stream[n]['time_stamps'][i] - eeg_stream[n]['time_stamps'][0]
                while (window_onset + window_size <= tmax):
                    onset.append(window_start + window_onset)
                    duration.append(window_size)
                    description.append(marker_stream[n]['time_series'][i][0].replace('task_', '').replace('-','/'))
                    window_onset = window_onset + window_size - window_overlap
        elif 'P300' in task:
            if('target' in marker_stream[n]['time_series'][i][0]):
                current_target = json.loads(marker_stream[n]['time_series'][i][0])['target']
            elif('flash' in marker_stream[n]['time_series'][i][0]):
                current_flash = json.loads(marker_stream[n]['time_series'][i][0])['flash']
                onset.append(marker_stream[n]['time_stamps'][i] - eeg_stream[n]['time_stamps'][0])
                duration.append(task_duration)
                description.append("target" if current_flash == current_target else "nontarget")
    annotations = mne.Annotations(onset, duration, description)
    eeg_raw = eeg_raw.set_annotations(annotations)
    
    # Create list of raw objects
    eeg_raw_list.append(eeg_raw)

(16, 31672)
Creating RawArray with float64 data, n_channels=16, n_times=31672
    Range : 0 ... 31671 =      0.000 ...   253.368 secs
Ready.
(16, 31922)
Creating RawArray with float64 data, n_channels=16, n_times=31922
    Range : 0 ... 31921 =      0.000 ...   255.368 secs
Ready.
(16, 31959)
Creating RawArray with float64 data, n_channels=16, n_times=31959
    Range : 0 ... 31958 =      0.000 ...   255.664 secs
Ready.
(16, 31316)
Creating RawArray with float64 data, n_channels=16, n_times=31316
    Range : 0 ... 31315 =      0.000 ...   250.520 secs
Ready.
(16, 31057)
Creating RawArray with float64 data, n_channels=16, n_times=31057
    Range : 0 ... 31056 =      0.000 ...   248.448 secs
Ready.


In [9]:
print(eeg_raw_list[0].annotations.orig_time)

None


In [10]:
# Concatenate raw objects
raw = mne.concatenate_raws(eeg_raw_list)
raw

<RawArray | 16 x 157926 (1263.4 s), ~19.3 MB, data loaded>

### Pre-processing

In [11]:
# Common average reference
raw_orig = raw.copy()
raw = raw.set_eeg_reference('average', projection=False)

if plotGraphs:
    fig = raw_orig.plot(title='Before Re-referencing', n_channels=16, scalings=scalings)
    fig = raw.plot(title='After Re-referencing', n_channels=16, scalings=scalings)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


In [12]:
# Bandpass filter data
raw_orig = raw.copy()
raw = raw.filter(l_freq=bp_l_freq, h_freq=bp_h_freq)

if plotGraphs:
    fig = raw_orig.plot(title='Before Filtering', scalings=scalings, duration=plot_duration)
    fig = raw.plot(title='After Filtering', scalings=scalings, duration=plot_duration)

Filtering raw data in 5 contiguous segments
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 413 samples (3.304 sec)



### Epoch data

In [13]:
# Epoch data
events, event_id = mne.events_from_annotations(raw, event_id=event_dict)
epochs = mne.Epochs(raw, events, event_id=event_id, tmin=0., tmax=window_size, baseline=None, picks='eeg', preload=True)
print(epochs)

Used Annotations descriptions: ['MI/push', 'rest']
Not setting metadata
Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated
Loading data for 100 events and 1001 original time points ...
0 bad epochs dropped
<Epochs |  100 events (all good), 0 - 8 sec, baseline off, ~12.2 MB, data loaded,
 'MI/push': 50
 'rest': 50>


### Features

In [14]:
# Labels
y = epochs.events[:,-1] - min(epochs.events[:,-1])
labels = y

### Classification

In [15]:
# Set up cross validation
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)

# Set up scoring
scoring = 'accuracy'
scores = {'Classifier': [],
          'Score': [],
          'Std': []
         }

In [16]:
w_len = np.concatenate((np.linspace(0.1, 2, 20), np.linspace(3., 8., 6)))
print(w_len)

[0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4 1.5 1.6 1.7 1.8
 1.9 2.  3.  4.  5.  6.  7.  8. ]


In [17]:
estimators = ["lwf", "oas"]
n_trials = epochs.get_data().shape[0]
dfc = list()

for wl in w_len:
    epochs = mne.Epochs(raw, events, event_id, tmin, tmin + wl, baseline=None, preload=True, verbose=False)
    for est in estimators:
        cov = Covariances(estimator=est).transform(epochs.get_data() * 1e6)
        for k in range(len(cov)):
            ev, _ = np.linalg.eigh(cov[k, :, :])
            dfc.append(dict(estimator=est, wlen=wl, cond=ev[-1] / ev[0]))
dfc = pd.DataFrame(dfc)

In [18]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.set(yscale="log")
sns.lineplot(data=dfc, x="wlen", y="cond", hue="estimator", ax=ax)
ax.set_title("Condition number of estimated covariance matrices")
ax.set_xlabel("Epoch length (s)")
ax.set_ylabel(r"$\lambda_{\max}$/$\lambda_{\min}$")
plt.tight_layout()

In [19]:
estimators = ["lwf", "oas"]
tmin = 0.0
n_trials, n_splits = n_trials = epochs.get_data().shape[0], 3
dfa = list()
sc = "balanced_accuracy"

classifiers = [('LR', LogisticRegression()),
               ('KNN2', KNeighborsClassifier(n_neighbors=2)),
               ('KNN3', KNeighborsClassifier(n_neighbors=3)),
               ('KNN4', KNeighborsClassifier(n_neighbors=4)),
               ('KNN5', KNeighborsClassifier(n_neighbors=5)),
               ('KNN6', KNeighborsClassifier(n_neighbors=6)),
               ('DT', DecisionTreeClassifier()),
               ('RF', RandomForestClassifier()),
               ('LDA', LinearDiscriminantAnalysis()),
               ('SVM', SVC())]

cv = StratifiedKFold(n_splits=n_splits, shuffle=True)
for wl in w_len:
    epochs = mne.Epochs(
        raw,
        events,
        event_id,
        tmin,
        tmin + wl,
        proj=True,
        preload=True,
        baseline=None,
        verbose=False,
    )
    X = epochs.get_data() * 1e6
    y = epochs.events[:,-1] - min(epochs.events[:,-1])
    for est in estimators:
        for clf_ in classifiers:
            clf = make_pipeline(Covariances(estimator=est), TSclassifier(clf=clf_[1]))

            try:
                score = cross_val_score(clf, X, y, cv=cv, scoring=sc)
                dfa += [dict(estimator=clf_[0]+'+'+est, wlen=wl, accuracy=sc) for sc in score]
            except ValueError:
                print(f"{est}: {wl} is not sufficent to estimate a SPD matrix")
                dfa += [dict(estimator=clf_[0]+'+'+est, wlen=wl, accuracy=np.nan)] * n_splits
dfa = pd.DataFrame(dfa)

In [20]:
dfa

,estimator,wlen,accuracy
0,LR+lwf,0.1,0.558824
1,LR+lwf,0.1,0.575368
2,LR+lwf,0.1,0.580882
3,KNN2+lwf,0.1,0.441176
4,KNN2+lwf,0.1,0.444853
...,...,...,...
1555,LDA+oas,8.0,0.636029
1556,LDA+oas,8.0,0.608456
1557,SVM+oas,8.0,0.617647
1558,SVM+oas,8.0,0.613971


In [22]:
fig, ax = plt.subplots(figsize=(6, 4))
sns.lineplot(
    data=dfa,
    x="wlen",
    y="accuracy",
    hue="estimator",
    style="estimator",
    ax=ax,
    ci=None,
    markers=True,
    dashes=False,
)
ax.set_title("Accuracy for different estimators and epoch lengths")
ax.set_xlabel("Epoch length (s)")
ax.set_ylabel("Classification accuracy")
plt.tight_layout()